In [97]:
#importing libraries
import sys
import time
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

In [98]:
#function for scrapping pharmeasy data
def scrape_pharmeasy_data(url):
    #write the scraping code for pharmeasy here
    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_info, error_obj = sys.exc_info()
        print('ERROR FOR LINK', url, '\n')
        print(error_type, ' Line: ', error_info.tb_lineno)

    time.sleep(2)

    soup = BeautifulSoup(page.text, 'html.parser')

    name = soup.find('div', {'class': 'MedicineOverviewSection_nameContainer__du_iv'}).text
    mrp = soup.find('span', {'class': 'PriceInfo_striked__Hk2U_'}).text
    marketer = soup.find('div', {'class': 'MedicineOverviewSection_brandName__rJFzE'}).text
    table = soup.find('table', {'class': 'DescriptionTable_seoTable__wKp77'})
    rows = table.find_all('tr')
    for row in rows:
        if row.find('td', {'class': 'DescriptionTable_field__l5jJ3'}).text == 'Offer Price':
            discounted_price = row.find('td', {'class': 'DescriptionTable_value__0GUMC'}).text
        elif row.find('td', {'class': 'DescriptionTable_field__l5jJ3'}).text == 'Contains':
            salts = row.find('td', {'class': 'DescriptionTable_value__0GUMC'}).text
            break
    else:
        salts = None

    url = url

    return name, mrp, discounted_price, marketer, salts, url

In [34]:
#function for scrapping netmeds data
def scrape_netmeds_data(url):
    #write the scraping code for netmeds here
    try:
        page = requests.get(url)
    except Exception as e:
        error_type, error_info, error_obj = sys.exc_info()
        print('ERROR FOR LINK', url, '\n')
        print(error_type, ' Line: ', error_info.tb_lineno)

    time.sleep(2)
    soup = BeautifulSoup(page.text, 'html.parser')
        
    name = soup.find('h1', {'class': 'black-txt'}).text
    mrp = soup.find('span', {'class': 'final-price'}).text
    discounted_price = soup.find('span', {'class': 'barBestPrice'}).text
    
    marketer_div = soup.find('div', string='Name of Manufacturer/Marketer')
    if marketer_div:
        marketer = marketer_div.find_next('div', {'class': 'manufacturer__name_value'}).text
    else:
        marketer = 'Not found'
    
    # Change the salts
    salts_div = soup.find('div', string='Ingredient')
    if salts_div:
        salts = salts_div.find_next('div', {'class': 'manufacturer__name_value'}).text
    else:
        salts = 'Not found'

    url = url

    return name, mrp, discounted_price, marketer, salts, url 

In [99]:
url = 'https://pharmeasy.in/online-medicine-order/browse?alphabet=b&page=0' 
response = requests.get(url)

if response.status_code == 200:
    html_content = response.text
else:
    print("Failed to fetch the webpage")

In [100]:
soup = BeautifulSoup(html_content, 'html.parser')

# Find all anchor tags with class 'BrowseList_medicine__cQZkc'
medicine_links = soup.find_all('a', class_='BrowseList_medicine__cQZkc')

# List to store extracted URLs
urls = []

# Extract URLs
for link in medicine_links:
    href = link.get('href')
    # Check if href starts with '/online-medicine-order/'
    if href.startswith('/online-medicine-order/'):
        # Construct the full URL and append it to the list
        full_url = f'https://pharmeasy.in{href}'
        urls.append(full_url)

# Function to scrape data from a given URL (You need to implement this function)
def scrape_pharmeasy_data(url):
    # Your scraping logic here
    pass

# List to store scraped data
data = []

# Iterate through the extracted URLs and scrape data
for url in urls:
    pharmeasy_data = scrape_pharmeasy_data(url)
    data.append(pharmeasy_data)

# At this point, 'data' should contain the scraped data from all the URLs
# You can then process this data as needed.


In [101]:
urls

['https://pharmeasy.in/online-medicine-order/b-29-strip-of-10-tablets-7181',
 'https://pharmeasy.in/online-medicine-order/b-active-strip-of-15-tablets-7782',
 'https://pharmeasy.in/online-medicine-order/b-fit-cd-strip-of-10-tablets-7248',
 'https://pharmeasy.in/online-medicine-order/baclof-10mg-tablet-13969',
 'https://pharmeasy.in/online-medicine-order/baclof-od-20mg-strip-of-10-tablets-14024',
 'https://pharmeasy.in/online-medicine-order/banatan-tube-of-50gm-cream-223555',
 'https://pharmeasy.in/online-medicine-order/basaglar-100iu-cartridge-of-3ml-injection-217391',
 'https://pharmeasy.in/online-medicine-order/basalog-100iu-injection-3ml-4794',
 'https://pharmeasy.in/online-medicine-order/basalog-refil-injection-3ml-4814',
 'https://pharmeasy.in/online-medicine-order/basalog-100iu-inj-5ml-4820',
 'https://pharmeasy.in/online-medicine-order/basalog-one-injection-pen-4857',
 'https://pharmeasy.in/online-medicine-order/becadexamin-cap-30-s-36928',
 'https://pharmeasy.in/online-medicine

In [104]:
data = []
for url in urls:
    output = scrape_pharmeasy_data(url)
    data.extend(output)

TypeError: 'NoneType' object is not iterable

In [103]:
output